In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

### Create Model

In [2]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride
        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
    def forward(self, x):
        x = F.relu(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        x = F.relu(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

# Train the network

In [3]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# Test the network

In [4]:
def test(model, device, test_loader,forTrain=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    if forTrain:
        print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    else:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

# Define main

In [5]:
def main(model,lr = 0.01,epochs = 10):
    # Training settings
    
    momentum = 0.5
    batch_size = 64
    test_batch_size = 1000
    log_interval = 10
    
    
    use_cuda = torch.cuda.is_available()                  # Check for CUDA.
    device = torch.device("cuda" if use_cuda else "cpu")  # Set which device to use.
    """
    
    """
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  
    
    
    """
    In order to make computations deterministic on your specific problem 
    on one specific platform
    """
    torch.manual_seed(1)  
    


    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval)
        test(model,device,train_loader,True)
        test(model, device, test_loader)


In [6]:
if __name__ == '__main__':
    model = Net()
    main(model)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309737
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.263927
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.202744
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.168830
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.052721
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.857354
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.582402
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.197256
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.803889
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.544675
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.522994
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.680057
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.575580
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.797604
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.423298
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.477571
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.366656
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.265494
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.322438
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.338928
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.206542
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.090168
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.077717
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.015429
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.102968
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.165266
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.058760
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.096534
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.021846
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.080900
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.030674
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.011366
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.015444
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.097621
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.027481
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.026014
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.094473
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.060311
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.256773
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.090990


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.016130
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.067150
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.013749
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.078068
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.020595
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.087219
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.095517
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.053045
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.062694
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.041220
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.096441
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.065508
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.103277
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.070114
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.007836
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.079215
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.076148
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.022859
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.014217
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.074369


Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.065880
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.007693
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.025931
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.032637
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.025185
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.007451
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.044005
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.005491
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.009995
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.009807
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.035785
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.005771
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.033956
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.048101
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.036326
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.005328
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.010784
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.061286
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.004862
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.058505
T

Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.044069
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.005668
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.067846
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.001689
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.014653
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.043654
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.006380
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.008418
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.003763
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.058248
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.055970

Train set: Average loss: 0.0222, Accuracy: 59607/60000 (99%)


Test set: Average loss: 0.0293, Accuracy: 9895/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.001587
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.015363
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.006898
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.012285
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.029721
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.031046
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.0

Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.015295
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.001770
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.027312
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.005754
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.032704
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.011509
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.002923
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.004301
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.003010
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.007176
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.013025
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.109473
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.014965
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.004945
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.083132
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.005377
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.046415
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.055650
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.002118
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.007214


# 4a Replace the ReLUs in the code with sigmoid units

### Create Model

In [7]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride
        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
    def forward(self, x):
        x = F.sigmoid(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        x = F.sigmoid(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

In [8]:
if __name__ == '__main__':
    model = Net()
    main(model)

/home/cs18mtech11033/venv/lib/python3.5/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320631
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.291723
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.289378
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.310559
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.295405
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.301975
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292058
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303689
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.294195
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.299564
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.286919
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.326697
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.319926
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.281132
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.296099
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.304295
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.300886
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.303195
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.283406
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.325223
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.546110
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.745385
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.331869
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.244992
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.528454
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.426207
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.472423
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.534139
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.407052
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.468634
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.245966
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.246363
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.227140
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.389153
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.464799
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.480817
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.405977
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.435840
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.624497
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.596631


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.297553
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.271870
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.210753
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.285341
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.298038
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.387125
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.463482
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.238686
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.346200
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.310720
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.425913
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.253640
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.390170
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.275868
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.124415
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.162852
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.416281
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.240308
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.139282
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.239748


Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.253707
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.121952
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.134035
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.124050
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.160885
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.134987
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.089120
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.133352
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.130247
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.221103
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.209091
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.079781
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.158369
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.116098
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.223595
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.156991
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.111515
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.306761
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.167478
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.109645
T

Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.127649
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.213305
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.214252
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.041610
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.210253
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.076432
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.135273
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.079743
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.063167
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.188618
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.171655

Train set: Average loss: 0.1377, Accuracy: 57507/60000 (96%)


Test set: Average loss: 0.1263, Accuracy: 9610/10000 (96%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.043752
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.102064
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.098334
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.110083
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.186578
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.138715
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.1

Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.091147
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.033082
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.161435
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.076737
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.103533
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.134909
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.068330
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.089669
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.153680
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.096188
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.153688
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.026775
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.100012
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.071069
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.072972
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.048291
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.184022
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.284937
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.038742
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.153466


### Discuss the results (i.e., explain why one type of unit performs better than the other).
Sparse representations seem to be more beneficial than dense representations.

ReLu, gradient will be 0 because of which the weights will not get adjusted during descent. 
ReLu is less computationally expensive than sigmoid(as it computes exponential) because it involves simpler mathematical operations.

Major benefits of ReLUs are sparsity and a reduced likelihood of vanishing gradient.

The gradient of sigmoids becomes increasingly small as the absolute value of x increases. Vanishing Gradient problem.

The constant gradient of ReLUs results in faster learning.

Due to above reason ReLu works better than sigmoid in terms of accuracy and computation.

#### Train set: Average loss: 0.1025, Accuracy: 58147/60000 (97%)


#### Test set: Average loss: 0.0937, Accuracy: 9708/10000 (97%)

## 4b Compare the accuracy achieved when varying the level of dropout in a CNN.

In [11]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self,prob):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.dropout1 = nn.Dropout2d(p=prob)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride
        self.dropout2 = nn.Dropout2d(p=prob)

        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
        
        
    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        x = self.dropout1(x)
        x = torch.sigmoid(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

In [12]:
#For p=0.25 dropout setting
if __name__ == '__main__':
    model = Net(0.25)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=0.25)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309660
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.268261
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300511
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.313956
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.319438
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.307716
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.283048
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.300055
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.307672
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.315037
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.296246
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.324444
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.340616
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.294051
Train Epoch:

Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.724224
Train Epoch: 2 [40960/60000 (68%)]	Loss: 0.961167
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.843117
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.544411
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.694605
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.715756
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.864826
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.968303
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.575961
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.529124
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.721045
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.599232
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.618780
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.785789
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.638731
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.698196
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.383410
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.534642
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.464664
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.637385


Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.352736
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.303619
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.436963
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.309868
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.286342
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.329139
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.325841
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.376442
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.334914
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.374621
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.359515
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.531350
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.631658
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.356526
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.479002
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.403614
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.429990
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.359161
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.549970
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.442887


Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.305885
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.236986
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.137017
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.233288
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.265353
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.307498
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.284977
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.345731
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.210066
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.241713
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.195726
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.261012
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.206485
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.119590
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.135958
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.183526
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.300389
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.327286
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.168901
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.223589
Train Epo

Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.128932
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.168139
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.127761
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.146896
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.201044
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.253468
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.173386
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.290709
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.275349
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.056674
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.326006
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.118844
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.254959
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.155450
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.161096
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.204162
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.272932

Train set: Average loss: 0.1483, Accuracy: 57292/60000 (95%)


Test set: Average loss: 0.1365, Accuracy: 9567/10000 (96%)

Train Epoch: 8 [0/60000 (0

Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.168837
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.120508
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.213528
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.197139
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.270382
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.095867
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.158895
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.089369
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.154063
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.197147
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.162859
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.235877
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.095819
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.209495
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.178792
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.154011
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.227687
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.092071
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.161523
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.158323


In [13]:
#For p=0.5 dropout setting
if __name__ == '__main__':
    model = Net(0.5)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.5)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=0.5)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313280
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.270594
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298709
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.308576
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.318385
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.302467
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.284776
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.306374
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.306078
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.307911
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.293015
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.327472
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.339641
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.295197
Train Epoch: 1

Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.780727
Train Epoch: 2 [40960/60000 (68%)]	Loss: 1.071516
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.954686
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.617680
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.756868
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.798823
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.924340
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.040548
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.651074
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.553479
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.790386
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.655799
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.695527
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.800940
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.691264
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.736217
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.444503
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.648978
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.513039
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.700763


Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.375203
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.352784
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.451945
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.337035
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.285636
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.319473
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.353974
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.418160
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.363112
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.402445
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.391728
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.521116
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.646996
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.399135
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.516115
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.422497
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.513623
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.396042
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.528343
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.439367


Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.293984
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.277664
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.127718
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.230255
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.263174
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.321532
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.263414
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.385982
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.226594
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.272193
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.207938
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.266842
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.251593
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.142715
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.143627
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.190013
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.319217
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.328154
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.216904
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.252488
Train Epo

Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.152316
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.196605
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.159289
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.161822
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.215011
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.264675
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.174550
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.303675
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.321387
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.055002
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.360079
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.152863
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.281551
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.178079
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.160444
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.180629
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.288393

Train set: Average loss: 0.1537, Accuracy: 57196/60000 (95%)


Test set: Average loss: 0.1416, Accuracy: 9554/10000 (96%)

Train Epoch: 8 [0/60000 (0

Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.194164
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.155425
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.206567
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.204279
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.278108
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.104570
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.188835
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.105205
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.181876
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.189735
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.154419
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.241238
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.103976
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.234244
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.191267
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.171187
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.242586
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.094032
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.149905
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.208406


In [14]:
# For 0.75
if __name__ == '__main__':
    model = Net(0.75)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.75)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=0.75)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311260
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.265829
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296021
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.311366
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.314372
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.300913
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.283533
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.302491
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.306874
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.307722
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.300328
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.335227
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.339448
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.293198
Train Epoch:

Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.978745
Train Epoch: 2 [40960/60000 (68%)]	Loss: 1.336712
Train Epoch: 2 [41600/60000 (69%)]	Loss: 1.126412
Train Epoch: 2 [42240/60000 (70%)]	Loss: 1.028088
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.957669
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.980018
Train Epoch: 2 [44160/60000 (74%)]	Loss: 1.241357
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.289618
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.957183
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.807541
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.931768
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.828169
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.982238
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.017226
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.925064
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.959708
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.651476
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.814394
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.645640
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.925098


Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.442755
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.416917
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.520895
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.409600
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.438592
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.463223
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.418548
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.513397
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.446024
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.469034
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.484156
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.657856
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.876433
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.552439
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.506023
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.534265
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.571862
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.379441
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.541708
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.551468


Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.303152
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.345317
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.150790
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.266756
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.390793
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.382137
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.426054
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.533144
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.305770
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.354293
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.252473
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.341802
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.342668
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.220513
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.208841
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.255914
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.447208
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.411479
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.282498
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.298713
Train Epo

Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.177613
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.268307
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.178294
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.237708
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.274569
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.286725
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.227758
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.325816
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.396475
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.083747
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.443449
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.156694
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.354144
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.210418
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.303475
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.283108
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.269687

Train set: Average loss: 0.1679, Accuracy: 56995/60000 (95%)


Test set: Average loss: 0.1561, Accuracy: 9505/10000 (95%)

Train Epoch: 8 [0/60000 (0

Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.165383
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.204045
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.270908
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.257637
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.359536
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.166481
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.237892
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.275273
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.288242
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.202242
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.165226
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.245793
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.285122
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.310639
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.242067
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.208145
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.370090
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.162379
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.180641
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.242546


In [15]:
# For p=1 dropout 
if __name__ == '__main__':
    model = Net(1)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=1)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=1)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319769
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295152
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.305737
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.315180
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.315212
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.284689
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.302469
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.302503
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.303984
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.313661
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.286101
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.341384
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.318566
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.304585
Train Epoch: 1 [89

Train Epoch: 2 [40320/60000 (67%)]	Loss: 2.273026
Train Epoch: 2 [40960/60000 (68%)]	Loss: 2.314878
Train Epoch: 2 [41600/60000 (69%)]	Loss: 2.306009
Train Epoch: 2 [42240/60000 (70%)]	Loss: 2.298081
Train Epoch: 2 [42880/60000 (71%)]	Loss: 2.305557
Train Epoch: 2 [43520/60000 (72%)]	Loss: 2.313088
Train Epoch: 2 [44160/60000 (74%)]	Loss: 2.295809
Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.305946
Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.317445
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.296377
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.304276
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.300325
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.309280
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.285239
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.313093
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.307829
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.298958
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.292790
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.307244
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.309647


Train Epoch: 4 [23040/60000 (38%)]	Loss: 2.302894
Train Epoch: 4 [23680/60000 (39%)]	Loss: 2.307900
Train Epoch: 4 [24320/60000 (41%)]	Loss: 2.289744
Train Epoch: 4 [24960/60000 (42%)]	Loss: 2.315331
Train Epoch: 4 [25600/60000 (43%)]	Loss: 2.299935
Train Epoch: 4 [26240/60000 (44%)]	Loss: 2.300179
Train Epoch: 4 [26880/60000 (45%)]	Loss: 2.296463
Train Epoch: 4 [27520/60000 (46%)]	Loss: 2.298573
Train Epoch: 4 [28160/60000 (47%)]	Loss: 2.298034
Train Epoch: 4 [28800/60000 (48%)]	Loss: 2.307000
Train Epoch: 4 [29440/60000 (49%)]	Loss: 2.292762
Train Epoch: 4 [30080/60000 (50%)]	Loss: 2.311874
Train Epoch: 4 [30720/60000 (51%)]	Loss: 2.312681
Train Epoch: 4 [31360/60000 (52%)]	Loss: 2.304702
Train Epoch: 4 [32000/60000 (53%)]	Loss: 2.302310
Train Epoch: 4 [32640/60000 (54%)]	Loss: 2.301719
Train Epoch: 4 [33280/60000 (55%)]	Loss: 2.296296
Train Epoch: 4 [33920/60000 (57%)]	Loss: 2.302996
Train Epoch: 4 [34560/60000 (58%)]	Loss: 2.299434
Train Epoch: 4 [35200/60000 (59%)]	Loss: 2.317970


Train Epoch: 6 [5120/60000 (9%)]	Loss: 2.288993
Train Epoch: 6 [5760/60000 (10%)]	Loss: 2.308336
Train Epoch: 6 [6400/60000 (11%)]	Loss: 2.284168
Train Epoch: 6 [7040/60000 (12%)]	Loss: 2.306086
Train Epoch: 6 [7680/60000 (13%)]	Loss: 2.323247
Train Epoch: 6 [8320/60000 (14%)]	Loss: 2.298598
Train Epoch: 6 [8960/60000 (15%)]	Loss: 2.308721
Train Epoch: 6 [9600/60000 (16%)]	Loss: 2.299620
Train Epoch: 6 [10240/60000 (17%)]	Loss: 2.318290
Train Epoch: 6 [10880/60000 (18%)]	Loss: 2.286944
Train Epoch: 6 [11520/60000 (19%)]	Loss: 2.302169
Train Epoch: 6 [12160/60000 (20%)]	Loss: 2.303167
Train Epoch: 6 [12800/60000 (21%)]	Loss: 2.299060
Train Epoch: 6 [13440/60000 (22%)]	Loss: 2.302670
Train Epoch: 6 [14080/60000 (23%)]	Loss: 2.294043
Train Epoch: 6 [14720/60000 (25%)]	Loss: 2.302921
Train Epoch: 6 [15360/60000 (26%)]	Loss: 2.292955
Train Epoch: 6 [16000/60000 (27%)]	Loss: 2.297726
Train Epoch: 6 [16640/60000 (28%)]	Loss: 2.300831
Train Epoch: 6 [17280/60000 (29%)]	Loss: 2.298800
Train Epo

Train Epoch: 7 [49280/60000 (82%)]	Loss: 2.295202
Train Epoch: 7 [49920/60000 (83%)]	Loss: 2.309532
Train Epoch: 7 [50560/60000 (84%)]	Loss: 2.303332
Train Epoch: 7 [51200/60000 (85%)]	Loss: 2.292075
Train Epoch: 7 [51840/60000 (86%)]	Loss: 2.309456
Train Epoch: 7 [52480/60000 (87%)]	Loss: 2.303700
Train Epoch: 7 [53120/60000 (88%)]	Loss: 2.299175
Train Epoch: 7 [53760/60000 (90%)]	Loss: 2.311823
Train Epoch: 7 [54400/60000 (91%)]	Loss: 2.297686
Train Epoch: 7 [55040/60000 (92%)]	Loss: 2.300937
Train Epoch: 7 [55680/60000 (93%)]	Loss: 2.316114
Train Epoch: 7 [56320/60000 (94%)]	Loss: 2.307621
Train Epoch: 7 [56960/60000 (95%)]	Loss: 2.296219
Train Epoch: 7 [57600/60000 (96%)]	Loss: 2.295900
Train Epoch: 7 [58240/60000 (97%)]	Loss: 2.292484
Train Epoch: 7 [58880/60000 (98%)]	Loss: 2.305221
Train Epoch: 7 [59520/60000 (99%)]	Loss: 2.291406

Train set: Average loss: 2.3017, Accuracy: 6742/60000 (11%)


Test set: Average loss: 2.3015, Accuracy: 1135/10000 (11%)

Train Epoch: 8 [0/60000 (0%

Train Epoch: 9 [32000/60000 (53%)]	Loss: 2.300533
Train Epoch: 9 [32640/60000 (54%)]	Loss: 2.312591
Train Epoch: 9 [33280/60000 (55%)]	Loss: 2.304087
Train Epoch: 9 [33920/60000 (57%)]	Loss: 2.299571
Train Epoch: 9 [34560/60000 (58%)]	Loss: 2.298976
Train Epoch: 9 [35200/60000 (59%)]	Loss: 2.299748
Train Epoch: 9 [35840/60000 (60%)]	Loss: 2.301102
Train Epoch: 9 [36480/60000 (61%)]	Loss: 2.289326
Train Epoch: 9 [37120/60000 (62%)]	Loss: 2.304114
Train Epoch: 9 [37760/60000 (63%)]	Loss: 2.297823
Train Epoch: 9 [38400/60000 (64%)]	Loss: 2.310640
Train Epoch: 9 [39040/60000 (65%)]	Loss: 2.312390
Train Epoch: 9 [39680/60000 (66%)]	Loss: 2.318609
Train Epoch: 9 [40320/60000 (67%)]	Loss: 2.304996
Train Epoch: 9 [40960/60000 (68%)]	Loss: 2.288674
Train Epoch: 9 [41600/60000 (69%)]	Loss: 2.304526
Train Epoch: 9 [42240/60000 (70%)]	Loss: 2.299304
Train Epoch: 9 [42880/60000 (71%)]	Loss: 2.298294
Train Epoch: 9 [43520/60000 (72%)]	Loss: 2.297251
Train Epoch: 9 [44160/60000 (74%)]	Loss: 2.300040


### Dropouts
Regularization is way to prevent over-fitting. Regularization reduces over-fitting by adding a penalty to the loss function. Dropout is an approach to regularization in neural networks which helps reducing interdependent learning amongst the neurons. Like in some layer only particular neurons are getting activated not others thats why we use dropout so that other neurons also gets activated and helps in generalization. Dropout has the effect of making the training process noisy, forcing nodes within a layer to probabilistically take on more or less responsibility for the inputs.

### For p = 0.25 This one is the best dropout for the above setting as it gives better performance over others because it is taking 75% of neuron into consideration.

Train set: Average loss: 0.1126, Accuracy: 57977/60000 (97%)

Test set: Average loss: 0.1039, Accuracy: 9665/10000 (97%)


### For p = .5 This one is comparable to 0.25 not of a much difference.

Train set: Average loss: 0.1174, Accuracy: 57852/60000 (96%)

Test set: Average loss: 0.1080, Accuracy: 9652/10000 (97%)

### For p = .75 This one is also comparable as there is not of a much difference in accuracy.

Train set: Average loss: 0.1294, Accuracy: 57639/60000 (96%)

Test set: Average loss: 0.1194, Accuracy: 9612/10000 (96%)



### For p = 1 This one is bad of above as it is giving 11% accuracy due to dropout(p=1) as all the neurons are deactivated.

Train set: Average loss: 2.3014, Accuracy: 6742/60000 (11%)


Test set: Average loss: 2.3012, Accuracy: 1135/10000 (11%)


## 4c Include batch normalization in your ReLU model with the best DropOut setting that you achieved.

### Model`

In [11]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self,prob):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.dropout1 = nn.Dropout2d(p=prob)    #DropOut with probability p
        self.bn1 = nn.BatchNorm2d(20)

        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride
        self.dropout2 = nn.Dropout(p=prob)    #DropOut with probability p
        
        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.bn2 = nn.BatchNorm1d(num_features=500)
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
    def forward(self, x):
        x = F.relu(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        #x = self.dropout1(x, training=self.training)
        x = self.dropout1(x)
        x = self.bn1(x)
        x = F.relu(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = self.dropout2(x)                    #Dropout on FC
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.bn2(x)
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

In [12]:
if __name__ == '__main__':
    model = Net(0.25)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25)
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout(p=0.25)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (bn2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.363844
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.806083
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.365599
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.164678
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.920309
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.767391
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.845909
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.772637
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.483847
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.498596
Train Epoch: 1 [6400/60000 (11%

Train Epoch: 2 [37760/60000 (63%)]	Loss: 0.054114
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.097166
Train Epoch: 2 [39040/60000 (65%)]	Loss: 0.050265
Train Epoch: 2 [39680/60000 (66%)]	Loss: 0.096951
Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.040320
Train Epoch: 2 [40960/60000 (68%)]	Loss: 0.054953
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.123329
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.087546
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.048073
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.130727
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.186550
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.161857
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.060818
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.046121
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.211005
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.109259
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.107943
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.155897
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.033416
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.105049


Train Epoch: 4 [20480/60000 (34%)]	Loss: 0.049736
Train Epoch: 4 [21120/60000 (35%)]	Loss: 0.034551
Train Epoch: 4 [21760/60000 (36%)]	Loss: 0.089971
Train Epoch: 4 [22400/60000 (37%)]	Loss: 0.097289
Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.066600
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.011917
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.123287
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.026274
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.029336
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.062053
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.045690
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.024283
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.028768
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.019958
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.018588
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.073063
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.118402
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.122407
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.105128
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.067356


Train Epoch: 6 [2560/60000 (4%)]	Loss: 0.027190
Train Epoch: 6 [3200/60000 (5%)]	Loss: 0.127785
Train Epoch: 6 [3840/60000 (6%)]	Loss: 0.016243
Train Epoch: 6 [4480/60000 (7%)]	Loss: 0.074205
Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.068476
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.033925
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.014621
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.033693
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.071027
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.071220
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.045308
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.109446
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.021202
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.053607
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.019454
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.030822
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.010834
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.081519
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.034869
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.049391
Train Epoch: 6 [1

Train Epoch: 7 [46720/60000 (78%)]	Loss: 0.094628
Train Epoch: 7 [47360/60000 (79%)]	Loss: 0.019515
Train Epoch: 7 [48000/60000 (80%)]	Loss: 0.014703
Train Epoch: 7 [48640/60000 (81%)]	Loss: 0.018032
Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.015236
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.060951
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.010016
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.121535
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.022444
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.020722
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.028743
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.057230
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.024657
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.010214
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.056805
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.008038
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.059930
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.042574
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.015891
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.017947


Train Epoch: 9 [29440/60000 (49%)]	Loss: 0.009445
Train Epoch: 9 [30080/60000 (50%)]	Loss: 0.028091
Train Epoch: 9 [30720/60000 (51%)]	Loss: 0.007258
Train Epoch: 9 [31360/60000 (52%)]	Loss: 0.010774
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.026296
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.035124
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.007217
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.031758
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.037860
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.010673
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.024593
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.005442
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.019637
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.021372
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.019381
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.017887
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.008871
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.009156
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.011832
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.022027


### Report
Due to the above Regularization method both are giving really good results around 99.33% on test and approx 100% on train. BatchNorm help in not overshoot of weights due to gradient.

Batch normalization reduces the amount by what the hidden unit values shift around (covariance shift, as distribution changes).

Batch normalization allows each layer of a network to learn by itself a little bit more independently of other layers.

Train set: Average loss: 0.0143, Accuracy: 59754/60000 (100%)


Test set: Average loss: 0.0207, Accuracy: 9933/10000 (99%)


### With Removed Dropout from above setting

In [13]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride
        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.bn2 = nn.BatchNorm1d(num_features=500)
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        x = self.bn1(x)
        x = F.relu(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.bn2(x)
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

In [14]:
if __name__ == '__main__':
    model = Net()
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (bn2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.382238
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.138992
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.714119
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.714034
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.468062
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.391578
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.494414
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.452533
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.224246
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.240156
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.266354
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.33

Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.028899
Train Epoch: 2 [39040/60000 (65%)]	Loss: 0.026582
Train Epoch: 2 [39680/60000 (66%)]	Loss: 0.060980
Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.015984
Train Epoch: 2 [40960/60000 (68%)]	Loss: 0.027581
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.049144
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.075601
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.045136
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.062856
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.114852
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.052742
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.045398
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.011511
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.123410
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.108684
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.028533
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.076906
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.013616
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.066277
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.038227


Train Epoch: 4 [21120/60000 (35%)]	Loss: 0.007705
Train Epoch: 4 [21760/60000 (36%)]	Loss: 0.038638
Train Epoch: 4 [22400/60000 (37%)]	Loss: 0.014955
Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.049955
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.005785
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.055868
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.009218
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.017688
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.006332
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.017256
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.015737
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.005935
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.026395
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.005917
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.042995
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.041623
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.051452
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.068334
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.004956
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.047708


Train Epoch: 6 [3200/60000 (5%)]	Loss: 0.012749
Train Epoch: 6 [3840/60000 (6%)]	Loss: 0.004957
Train Epoch: 6 [4480/60000 (7%)]	Loss: 0.024485
Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.012694
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.012554
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.003573
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.013058
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.009770
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.011792
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.009850
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.032745
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.005992
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.064350
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.014607
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.036522
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.002549
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.009014
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.004467
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.007814
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.003573
Train Epoch: 6 

Train Epoch: 7 [47360/60000 (79%)]	Loss: 0.010031
Train Epoch: 7 [48000/60000 (80%)]	Loss: 0.002902
Train Epoch: 7 [48640/60000 (81%)]	Loss: 0.001961
Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.012562
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.012095
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.005107
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.030398
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.015646
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.009232
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.006027
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.010101
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.009564
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.002792
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.013476
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.009248
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.015865
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.016815
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.002636
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.008989
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.008975


Train Epoch: 9 [30080/60000 (50%)]	Loss: 0.007475
Train Epoch: 9 [30720/60000 (51%)]	Loss: 0.001862
Train Epoch: 9 [31360/60000 (52%)]	Loss: 0.001430
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.012854
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.005030
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.004535
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.005789
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.013905
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.004373
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.010113
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.001828
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.016000
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.003685
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.021891
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.008047
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.005801
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.005573
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.002120
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.004314
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.005183


### Report
Similar to dropout, it adds some noise to each hidden layer’s activations. Therefore, if we use batch normalization, we will use less dropout, which is a good thing because we are not going to lose a lot of information. Since we are using two regularizer in above setting we are getting better result than above setting.

Train set: Average loss: 0.0057, Accuracy: 59959/60000 (100%)


Test set: Average loss: 0.0229, Accuracy: 9928/10000 (99%)

## 4d With ReLU units and batch normalization (no DropOut).

### Init Weights

In [8]:
def init_weights(m):
#     print(m)
    if isinstance(m,nn.Linear):
        #m.weight.data.fill_(1.0)
        torch.nn.init.xavier_uniform_(m.weight)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
#        m.weight.data.fill_(1.0)
#         if m.bias:
#             torch.nn.init.xavier_uniform_(m.bias)

### Model

In [9]:
class Net(nn.Module):                           #Make a class to create a Model
    def __init__(self,prob):                         #Init the class with the layer we want out model to have.
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)     #Conv layer with 1 i/p channel,20 o/p channel, 5 Kernel Size, 1 stride
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)    #Conv layer with 20 i/p channel,50 o/p channel, 5 Kernel Size, 1 stride        
        self.fc1 = nn.Linear(4*4*50, 500)       #Linear layer with 400 fan_in and 500 fan_out
        self.bn2 = nn.BatchNorm1d(num_features=500)
        self.fc2 = nn.Linear(500, 10)           #Linear layer with 400 fan_in and 10 fan_out(i.e. number of output)
    def forward(self, x):
        x = F.relu(self.conv1(x))               #Applying relu on conv layer.
        x = F.max_pool2d(x, 2, 2)               #Applying pooling of size3 on output of conv.
        x = self.bn1(x)
        x = F.relu(self.conv2(x))               #Applying conv and relu on the output of maxpool.
        x = F.max_pool2d(x, 2, 2)               #Applying maxpool on the result of 2 conv layer.
        x = x.view(-1, 4*4*50)                  #Changing the dimension of image in 1d so that we can apply Linear layer. 
        x = F.relu(self.fc1(x))                 #Relu on first FC layer
        x = self.bn2(x)
        x = self.fc2(x)                         #Second FC Layer
        return F.log_softmax(x, dim=1)          #Applying softmax on the output to get the result which class output belong to.

In [10]:
if __name__ == '__main__':
    model = Net(0.5)
    model.apply(init_weights)
    print(model)
    main(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (bn2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.882009
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.838444
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.516683
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.551692
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.338606
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.350835
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.441517
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.468277
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.181623
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.175499
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.233012
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.25

Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.044912
Train Epoch: 2 [39040/60000 (65%)]	Loss: 0.025791
Train Epoch: 2 [39680/60000 (66%)]	Loss: 0.071492
Train Epoch: 2 [40320/60000 (67%)]	Loss: 0.017230
Train Epoch: 2 [40960/60000 (68%)]	Loss: 0.017498
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.090766
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.041435
Train Epoch: 2 [42880/60000 (71%)]	Loss: 0.052062
Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.056389
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.106541
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.087566
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.043183
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.025185
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.135360
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.100908
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.055704
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.075138
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.021841
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.072655
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.024628


Train Epoch: 4 [21120/60000 (35%)]	Loss: 0.009861
Train Epoch: 4 [21760/60000 (36%)]	Loss: 0.029929
Train Epoch: 4 [22400/60000 (37%)]	Loss: 0.021979
Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.030523
Train Epoch: 4 [23680/60000 (39%)]	Loss: 0.012005
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.093901
Train Epoch: 4 [24960/60000 (42%)]	Loss: 0.012530
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.018170
Train Epoch: 4 [26240/60000 (44%)]	Loss: 0.014820
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.026500
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.018826
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.006066
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.023016
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.013007
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.058672
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.083667
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.035280
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.059474
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.017273
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.044071


Train Epoch: 6 [3200/60000 (5%)]	Loss: 0.019529
Train Epoch: 6 [3840/60000 (6%)]	Loss: 0.007136
Train Epoch: 6 [4480/60000 (7%)]	Loss: 0.057661
Train Epoch: 6 [5120/60000 (9%)]	Loss: 0.012709
Train Epoch: 6 [5760/60000 (10%)]	Loss: 0.012850
Train Epoch: 6 [6400/60000 (11%)]	Loss: 0.004833
Train Epoch: 6 [7040/60000 (12%)]	Loss: 0.009845
Train Epoch: 6 [7680/60000 (13%)]	Loss: 0.048527
Train Epoch: 6 [8320/60000 (14%)]	Loss: 0.011463
Train Epoch: 6 [8960/60000 (15%)]	Loss: 0.035798
Train Epoch: 6 [9600/60000 (16%)]	Loss: 0.034512
Train Epoch: 6 [10240/60000 (17%)]	Loss: 0.005023
Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.051540
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.015792
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.046277
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.002758
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.020793
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.013994
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.013265
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.004678
Train Epoch: 6 

Train Epoch: 7 [47360/60000 (79%)]	Loss: 0.016013
Train Epoch: 7 [48000/60000 (80%)]	Loss: 0.009618
Train Epoch: 7 [48640/60000 (81%)]	Loss: 0.004106
Train Epoch: 7 [49280/60000 (82%)]	Loss: 0.018659
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.014089
Train Epoch: 7 [50560/60000 (84%)]	Loss: 0.012832
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.014367
Train Epoch: 7 [51840/60000 (86%)]	Loss: 0.007758
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.017840
Train Epoch: 7 [53120/60000 (88%)]	Loss: 0.009385
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.015148
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.012990
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.002112
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.038200
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.022151
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.014187
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.018546
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.003988
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.011597
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.017773


Train Epoch: 9 [30080/60000 (50%)]	Loss: 0.023135
Train Epoch: 9 [30720/60000 (51%)]	Loss: 0.001955
Train Epoch: 9 [31360/60000 (52%)]	Loss: 0.002621
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.008884
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.006718
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.005485
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.006693
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.020432
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.003799
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.021731
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.003178
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.035674
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.003700
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.015091
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.008607
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.007121
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.002922
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.002085
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.007750
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.004540


### Report
I did xavier_uniform on the weight which are already there. I also tried weight =1 in FC which gave really bad result like 66% with batchnorm. With uniform on all the layer accuracy is really good.
So it doesn't matter of batchnorm for different weight initialization.